## classwork 1

1. 实现pdf文档的导入，并对其进行分割，输出分割后的段数与某段的内容

2. 实现在线新闻的导入，并对其进行分割，输出分割后的段数与某段的内容

## CLassWork 2  
* 1.基于pdf文档的导入和分隔后，向量化对应文档并将其存入向量数据库并持久化（建立不同文档不同目录），然后通过相似性进行检索
* 2.对在线新闻进行同上述一样的操作，并使用相似性进行检索

In [1]:
import os
from dotenv import load_dotenv

# 从.env文件中 加载环境变量
load_dotenv()

# 加载大模型（聊天模型）
from langchain_community.chat_models import ChatZhipuAI
llm = ChatZhipuAI(
    temperature=0.3,
    model="glm-4-plus",
    zhipuai_api_key=os.getenv("ZHIPUAI_API_KEY"),
)

# RAG（Retrieval检索 + Generation生成）
# 数据准备阶段：数据提取（文档加载） 》〉 文本分割 》〉 向量化 》〉 数据入库（向量数据库存储 采用本地chroma数据库存储）
# pdf文档读入
from langchain_community.document_loaders import PyPDFLoader

file_path = "./data/2024年金融科技生态蓝皮书.pdf"
loader_pdf = PyPDFLoader(file_path)
docs_pdf = loader_pdf.load()

# pdf文本分隔
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter_pdf = RecursiveCharacterTextSplitter(
    chunk_size = 1000, 
    chunk_overlap = 200,
    length_function = len,
    separators= ["\n\n", "\n", " ", "."]
)

docs_pdf = text_splitter_pdf.split_documents(docs_pdf)

# print(docs_pdf[2]) #打印分割后的文本
# sum([len(doc.page_content) for doc in docs_pdf]) 计算字数

# 文档词向量化 与 向量数据库存储
#加载zhipuai embedding模型
from langchain_community.embeddings import ZhipuAIEmbeddings

embeddings_model =ZhipuAIEmbeddings(
    model='embedding-3',
    api_key=os.getenv("ZHIPUAI_API_KEY"),
)

# 因为emded_documents方法 调用的字符串列表 所以texts是将docs_pdf转化为字符串列表
# texts = [doc.page_content for doc in docs_pdf]
# # 向量化 其结果在内存中，临时性的
# embeddings = embeddings_model.embed_documents(texts)

# print(embeddings[0]) #打印向量化后的数据
# print(len(embeddings)) #打印向量化后的数据长度
# print(len(embeddings[0])) #打印向量化后的数据

# 向量数据库存储
from langchain.vectorstores import Chroma

# 该指令是创建向量数据库 这里的persist_directory是指向量数据库存储的路径
# vectordb = Chroma.from_documents( 
#     documents=docs_pdf,
#     embedding=embeddings_model,
#     persist_directory="./db/chroma_db",
# )

# 已有向量数据库 导入
vectordb_load = Chroma( 
    persist_directory="./db/chroma_db",
    embedding_function=embeddings_model,
)

# 通过 已有向量数据库 检索相关文档
res = vectordb_load.similarity_search("金融科技是什么？")
print(res[0].page_content) #打印检索到的相关文档


/var/folders/fq/3pmzyx3s60l0sjm0lfcpdfhh0000gn/T/ipykernel_2194/3858266800.py:67: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb_load = Chroma(


前言
2016年，国际权威组织金融稳定理事会（FSB）首次提出金融
科技是“由大数据、区块链、云计算、人工智能等新兴前沿技术带
动，对金融市场以及金融服务业务供给产生重大影响的新兴业务模
式、新技术应用、新产品服务等。 ”随着新兴前沿技术在金融业应用
加速，金融科技已成为科技创新在金融领域融合发展的集中体现。
中央金融工作会议指出，要做好科技金融、绿色金融、普惠金
融、养老金融、数字金融“五篇大文章” ，指明了金融支持经济高质
量发展的发力点，同时也给金融科技发展和应用指明方向。在新的
形势下，金融科技将进一步发挥赋能和支撑作用，提升金融业服务
于国民经济重点领域和薄弱环节，为高质量发展做出贡献。
从全球市场看，金融业数字化深入推进，金融科技成为国际竞
争合作新关注点，同时主要经济体对金融科技的监管和治理也呈现
新的方向，尤其是关注技术可靠性和对金融市场稳定的影响。在数
字技术持续推动下，金融科技应用下沉到更广泛群体，与多样化场
景结合推动嵌入式金融快速发展，同时更多关注ESG、普惠群体，
凸显社会价值。
金融科技的定位和导向更加清晰，发挥科技创新的引擎作用，
支持金融强国目标实现。从发展动力看，通过持续推动新技术在金
融业的应用，金融科技助力核心要素跃升，加速形成新质生产力；
从监管建设来看，在金融业“五大监管”落实过程中，监管数智化


In [4]:
import os
from dotenv import load_dotenv

# 从.env文件中 加载环境变量
load_dotenv()

# 加载大模型（聊天模型）
from langchain_community.chat_models import ChatZhipuAI
llm = ChatZhipuAI(
    temperature=0.3,
    model="glm-4-plus",
    zhipuai_api_key=os.getenv("ZHIPUAI_API_KEY"),
)

# RAG（Retrieval检索 + Generation生成）
# 数据准备阶段：数据提取（文档加载） 》〉 文本分割 》〉 向量化 》〉 数据入库（向量数据库存储 采用本地chroma数据库存储）

# web文件读入
from langchain_community.document_loaders import WebBaseLoader

# WEB_URL = "https://news.ifeng.com/c/8Y3TlIcTsj0"
WEB_URL = "https://www.zaobao.com/realtime/china/story20250513-6329785"
loader_html = WebBaseLoader(WEB_URL)
docs_html = loader_html.load()

# pdf文本分隔
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter_html = RecursiveCharacterTextSplitter(
    chunk_size = 1000, 
    chunk_overlap = 200,
    length_function = len,
    separators= ["\n\n", "\n", " ", "."]
)

docs_html = text_splitter_html.split_documents(docs_html)

# print(docs_html[2]) #打印分割后的文本
# sum([len(doc.page_content) for doc in docs_html]) 计算字数

# 文档词向量化 与 向量数据库存储
#加载zhipuai embedding模型
from langchain_community.embeddings import ZhipuAIEmbeddings

embeddings_model =ZhipuAIEmbeddings(
    model='embedding-3',
    api_key=os.getenv("ZHIPUAI_API_KEY"),
)

# 因为emded_documents方法 调用的字符串列表 所以texts是将docs_pdf转化为字符串列表
# texts = [doc.page_content for doc in docs_pdf]
# # 向量化 其结果在内存中，临时性的
# embeddings = embeddings_model.embed_documents(texts)

# print(embeddings[0]) #打印向量化后的数据
# print(len(embeddings)) #打印向量化后的数据长度
# print(len(embeddings[0])) #打印向量化后的数据

# 向量数据库存储
from langchain.vectorstores import Chroma

# 该指令是创建向量数据库 这里的persist_directory是指向量数据库存储的路径
# vectordb = Chroma.from_documents( 
#     documents=docs_html,
#     embedding=embeddings_model,
#     persist_directory="./db/chroma_db_web",
# )

# 已有向量数据库 导入
vectordb_load = Chroma( 
    persist_directory="./db/chroma_db_web",
    embedding_function=embeddings_model,
)

# 通过 已有向量数据库 检索相关文档
res = vectordb_load.similarity_search("美国对中国进口关税有什么变化（明确变化时间）？")
print(res[0].page_content) #打印检索到的相关文档


联合声明指出，中国将暂停或取消自4月2日起对美国实施的非关税反制措施。彭博社分析，这些措施主要是将七类重稀土相关产品纳入出口管制清单。华盛顿将解除这些限制视为优先任务之一，以缓解美国多个行业在供应链方面面临的压力。
特朗普星期一说：“日内瓦的谈判气氛非常友好，双方关系也很好。我们并不想伤害中国，中国此前已经受到了严重打击。”
受利好消息提振，标普500指数星期一上涨逾3%，债券、黄金及避险货币等防御型资产纷纷走弱。美元录得自去年11月美国总统大选反弹以来的最大单日涨幅。大型科技股强劲反弹，带动纳斯达克100指数重回牛市区间。
由中国国务院副总理何立峰率领的代表团在过去的周末与美国财长贝森特及贸易代表格里尔举行贸易谈判。根据双方发布的联合声明，两国将暂时降低关税，为接下来的深入谈判争取时间。
美国将原本对中国进口商品征收的145%关税调降至30%。仍保留的30%关税中，包括以芬太尼为由加征的20%关税，以及10%的基准关税；中国则把对美国加征的125%报复性关税，降至10%。
在中美达成90天贸易战休战协议后，美国总统特朗普说，中国已同意取消对美进口商品施加的非关税壁垒，并暗示若谈判取得进展，北京方面可能做出更大让步。



    延伸阅读
   


  
中美日内瓦会谈落幕 双方互降关税并将继续协商 



  
日内瓦会谈后互降关税 学者：华盛顿可能转向其他手段施压北京


特朗普没有具体说明中国承诺取消哪些壁垒，但他承认接下来的谈判只会更加复杂。他说：“需要时间来把口头承诺变成白纸黑字，你知道的，这可不是一件容易的事。”










相关热词


特朗普
中美贸易战
中美关系
关税
中美关税













相关推荐


中美贸易战休战后 美客户据报应更容易获得中国稀土许可
